In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

# Cargar el dataset
file_path = "../data/processed/train_data_processed_imputeKnn_scale.csv"
data = pd.read_csv(file_path)

# Inspección inicial
print("Dimensiones del dataset:", data.shape)
print(data.head())

# Identificar la variable objetivo y las características
target_col = "target"
features = [col for col in data.columns if col != target_col]

# Preprocesamiento: manejar valores categóricos y nulos
# Convertir categorías a valores numéricos
label_encoders = {}
for col in data.select_dtypes(include=['object']).columns:
    if col in features:
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col].astype(str))
        label_encoders[col] = le

# Dividir en características (X) y etiqueta (y)
X = data[features]
y = data[target_col]

# Dividir en conjuntos de características (X) y etiquetas (y)
X_array = X.values
y_array = y.values

# Parámetros para XGBoost (iniciales)
params = {
    "objective": "reg:squarederror",
    "eval_metric": "rmse",  # Métrica directa de RMSE
    "seed": 42,
}

# Establecer un espacio de búsqueda para la optimización de parámetros
param_grid = {
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [4, 6, 8],
    "min_child_weight": [1, 3, 5],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "reg_alpha": [0, 0.1, 0.5],
    "reg_lambda": [0.1, 0.5, 1.0],
}

# Implementación de K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmse_scores = []

# Búsqueda de hiperparámetros con GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb.XGBRegressor(objective='reg:squarederror', seed=42),
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=kf,
    verbose=1,
    n_jobs=-1
)

# Entrenamiento y optimización
grid_search.fit(X_array, y_array)

# Ver los mejores parámetros encontrados
print(f"Mejores parámetros encontrados: {grid_search.best_params_}")

# Obtener el mejor modelo
best_model = grid_search.best_estimator_

# Evaluar el modelo en el conjunto de validación
y_pred = best_model.predict(X_array)
rmse = np.sqrt(mean_squared_error(y_array, y_pred))
print(f"RMSE final con los mejores parámetros: {rmse:.4f}")


Dimensiones del dataset: (118917, 18)
        brand  che_pc_usd  che_perc_gdp corporation       country launch_date  \
0  BRAND_354E   -0.861595     -0.069532   CORP_D524  COUNTRY_88A3  2014-06-01   
1  BRAND_626D    1.012675      0.542469   CORP_01C7  COUNTRY_8B47  2014-06-01   
2  BRAND_45D9   -0.861595     -0.069532   CORP_39F7  COUNTRY_88A3  2014-06-01   
3  BRAND_D724    0.780386      1.333155   CORP_711A  COUNTRY_445D  2014-06-01   
4  BRAND_4887    0.626762      1.359908   CORP_443D  COUNTRY_D8B0  2014-06-01   

         date       drug_id                indication  insurance_perc_che  \
0  2014-06-01  DRUG_ID_8795              ['IND_C3B6']            1.150233   
1  2014-06-01  DRUG_ID_E66E  ['IND_1590', 'IND_ECAC']           -0.845488   
2  2014-06-01  DRUG_ID_F272              ['IND_B2EF']            1.150233   
3  2014-06-01  DRUG_ID_1D4E              ['IND_BAFB']           -1.332507   
4  2014-06-01  DRUG_ID_AA88              ['IND_3F31']            1.483735   

   populatio

KeyboardInterrupt: 